In [1]:
import pandas as pd
import numpy as np
n_thread=4

In [2]:
ls -lha ../data/

total 56G
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 27 16:53 ./
drwxrwxr-x 7 ubuntu ubuntu 4.0K Apr 27 18:29 ../
-rw-rw-r-- 1 ubuntu ubuntu 3.3G Apr 25 19:19 basic_table
-rw-rw-r-- 1 ubuntu ubuntu 850M Apr 25 19:19 channel_categorical
-rw-rw-r-- 1 ubuntu ubuntu 1.9G Apr 25 19:19 click_count_table
-rw-rw-r-- 1 ubuntu ubuntu 802M Apr 25 19:19 click_time_interval_ip_device_os_app_hash
-rw-rw-r-- 1 ubuntu ubuntu 706M Apr 26 07:28 down_sampling_train
-rw-rw-r-- 1 ubuntu ubuntu 3.2G Apr 25 19:19 intervals_table
-rw-rw-r-- 1 ubuntu ubuntu  10G Apr 25 19:19 misc_counts
-rw-rw-r-- 1 ubuntu ubuntu  16G Apr 25 19:19 multi_basic
-rw-rw-r-- 1 ubuntu ubuntu  12G Apr 25 21:45 next_prev_channels
-rw-rw-r-- 1 ubuntu ubuntu 2.3G Apr 25 19:20 ranks_table
-rw-rw-r-- 1 ubuntu ubuntu 5.0G Apr 26 06:08 time_of_day
-rw-rw-r-- 1 ubuntu ubuntu 778M Apr 25 19:22 time_table


In [3]:
click_time = pd.read_feather("../data/time_of_day", nthreads=n_thread)

In [5]:
click_time = click_time[["second_of_day"]]

In [6]:
click_time["dow"] = pd.read_feather("../data/time_table", nthreads=n_thread)["dow"]

/home/ubuntu/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
click_time["epoctime"] = click_time["second_of_day"] + click_time["dow"]*24*3600

/home/ubuntu/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
basic = pd.read_feather("../data/basic_table", nthreads=n_thread)

In [12]:
basic.drop(columns=["click_id", "is_attributed"], inplace=True)

In [14]:
df = pd.concat([basic, click_time], axis=1, copy=False)

In [16]:
del basic, click_time

In [18]:
from logzero import logger

In [11]:
def create_next_prev(df, column, second="epoctime"):
    logger.info("sorting...")
    df.sort_values(by=[column, second], inplace=True)
    logger.info("sorted")
    prev_column="prev_click_second_{}".format(column)
    next_column="next_click_second_{}".format(column)
    
    df[prev_column] = df[second] - df[second].shift(+1)
    df["tmp"] = df[column].shift(+1)
    df[prev_column].where(df["tmp"] == df[column], np.nan, inplace=True)
    logger.info("done prev")
    
    df[next_column] = df[second].shift(-1) - df[second]
    df["tmp"] = df[column].shift(-1)
    df[next_column].where(df["tmp"] == df[column], np.nan, inplace=True)
    logger.info("done next")

    df.drop(columns=["tmp"], inplace=True)
    logger.info("dropped")
    return df

In [26]:
df = create_next_prev(df, "ip")

[I 180428 03:52:46 <ipython-input-25-ddf238345cf0>:2] sorting...
[I 180428 03:56:24 <ipython-input-25-ddf238345cf0>:4] sorted
[I 180428 03:56:32 <ipython-input-25-ddf238345cf0>:11] done prev
[I 180428 03:56:39 <ipython-input-25-ddf238345cf0>:16] done next
[I 180428 03:57:07 <ipython-input-25-ddf238345cf0>:19] dropped


In [27]:
df[["ip", "prev_click_second_ip", "next_click_second_ip"]]

,ip,prev_click_second_ip,next_click_second_ip
187940170,0,NaN,93.0
187794271,0,93.0,NaN
137415147,1,NaN,105.0
137504305,1,105.0,7177.0
141459793,1,7177.0,3.0
141462609,1,3.0,0.0
141462919,1,0.0,1.0
141463810,1,1.0,0.0
141463515,1,0.0,5.0
141467628,1,5.0,24545.0


In [28]:
%%time
df = create_next_prev(df, "device")
df = create_next_prev(df, "app")
df = create_next_prev(df, "channel")
df = create_next_prev(df, "os")

[I 180428 03:57:57 <ipython-input-25-ddf238345cf0>:2] sorting...
[I 180428 03:59:04 <ipython-input-25-ddf238345cf0>:4] sorted
[I 180428 03:59:12 <ipython-input-25-ddf238345cf0>:11] done prev
[I 180428 03:59:19 <ipython-input-25-ddf238345cf0>:16] done next
[I 180428 03:59:46 <ipython-input-25-ddf238345cf0>:19] dropped
[I 180428 03:59:46 <ipython-input-25-ddf238345cf0>:2] sorting...
[I 180428 04:00:49 <ipython-input-25-ddf238345cf0>:4] sorted
[I 180428 04:00:57 <ipython-input-25-ddf238345cf0>:11] done prev
[I 180428 04:01:04 <ipython-input-25-ddf238345cf0>:16] done next
[I 180428 04:01:31 <ipython-input-25-ddf238345cf0>:19] dropped
[I 180428 04:01:31 <ipython-input-25-ddf238345cf0>:2] sorting...
[I 180428 04:02:27 <ipython-input-25-ddf238345cf0>:4] sorted
[I 180428 04:02:34 <ipython-input-25-ddf238345cf0>:11] done prev
[I 180428 04:02:41 <ipython-input-25-ddf238345cf0>:16] done next
[I 180428 04:03:09 <ipython-input-25-ddf238345cf0>:19] dropped
[I 180428 04:03:09 <ipython-input-25-ddf238

CPU times: user 5min, sys: 2min 17s, total: 7min 18s
Wall time: 7min 18s


In [29]:
df.columns

Index(['app', 'channel', 'device', 'ip', 'os', 'second_of_day', 'dow',
       'epoctime', 'prev_click_second_ip', 'next_click_second_ip',
       'prev_click_second_device', 'next_click_second_device',
       'prev_click_second_app', 'next_click_second_app',
       'prev_click_second_channel', 'next_click_second_channel',
       'prev_click_second_os', 'next_click_second_os'],
      dtype='object')

In [32]:
df.sort_index(inplace=True)

In [33]:
writing = ['prev_click_second_ip', 'next_click_second_ip',
       'prev_click_second_device', 'next_click_second_device',
       'prev_click_second_app', 'next_click_second_app',
       'prev_click_second_channel', 'next_click_second_channel',
       'prev_click_second_os', 'next_click_second_os']
df[writing].to_feather("../data/prev_next_click_basic")

In [34]:
df = df[["dow", "epoctime"]]

In [35]:
import gc
gc.collect()

3158

In [2]:
df = pd.read_feather("../data/multi_basic")

In [3]:
df["dow"] = pd.read_feather("../data/time_table", nthreads=n_thread)["dow"]

In [4]:
df["second_of_day"] = pd.read_feather("../data/time_of_day", nthreads=n_thread)["second_of_day"]

In [5]:
df["epoctime"] = df["dow"]*24*3600 + df["second_of_day"]

In [6]:
df.drop(columns=["second_of_day"], inplace=True)

In [7]:
df.columns

Index(['ip_os', 'ip_device', 'ip_app', 'ip_channel', 'os_device', 'os_app',
       'os_channel', 'device_app', 'device_channel', 'app_channel',
       'ip_os_device', 'ip_os_app', 'ip_os_channel', 'ip_device_app',
       'ip_device_channel', 'ip_app_channel', 'os_device_app',
       'os_device_channel', 'os_app_channel', 'device_app_channel',
       'ip_os_device_app', 'ip_os_device_channel', 'ip_os_app_channel',
       'ip_device_app_channel', 'os_device_app_channel',
       'ip_os_device_app_channel', 'dow', 'epoctime'],
      dtype='object')

In [9]:
from logzero import logger

In [12]:
for two in ['ip_os', 'ip_device', 'ip_app', 'ip_channel',
       'os_device', 'os_app', 'os_channel', 'device_app', 'device_channel',
       'app_channel']:
    logger.info(two)
    df = create_next_prev(df, two)

[I 180428 04:41:59 <ipython-input-12-2fde797bbec5>:4] ip_os
[I 180428 04:41:59 <ipython-input-11-ddf238345cf0>:2] sorting...
[I 180428 04:47:30 <ipython-input-11-ddf238345cf0>:4] sorted
[I 180428 04:47:37 <ipython-input-11-ddf238345cf0>:11] done prev
[I 180428 04:47:45 <ipython-input-11-ddf238345cf0>:16] done next
[I 180428 04:48:10 <ipython-input-11-ddf238345cf0>:19] dropped
[I 180428 04:48:10 <ipython-input-12-2fde797bbec5>:4] ip_device
[I 180428 04:48:10 <ipython-input-11-ddf238345cf0>:2] sorting...
[I 180428 04:50:02 <ipython-input-11-ddf238345cf0>:4] sorted
[I 180428 04:50:09 <ipython-input-11-ddf238345cf0>:11] done prev
[I 180428 04:50:17 <ipython-input-11-ddf238345cf0>:16] done next
[I 180428 04:50:46 <ipython-input-11-ddf238345cf0>:19] dropped
[I 180428 04:50:46 <ipython-input-12-2fde797bbec5>:4] ip_app
[I 180428 04:50:46 <ipython-input-11-ddf238345cf0>:2] sorting...
[I 180428 04:53:10 <ipython-input-11-ddf238345cf0>:4] sorted
[I 180428 04:53:17 <ipython-input-11-ddf238345cf0>:

In [14]:
df.drop(columns=['ip_os', 'ip_device', 'ip_app', 'ip_channel', 'os_device', 'os_app',
       'os_channel', 'device_app', 'device_channel', 'app_channel',
       'ip_os_device', 'ip_os_app', 'ip_os_channel', 'ip_device_app',
       'ip_device_channel', 'ip_app_channel', 'os_device_app',
       'os_device_channel', 'os_app_channel', 'device_app_channel',
       'ip_os_device_app', 'ip_os_device_channel', 'ip_os_app_channel',
       'ip_device_app_channel', 'os_device_app_channel',
       'ip_os_device_app_channel'], inplace=True)

In [16]:
df.drop(columns=["dow", "epoctime"], inplace=True)

In [19]:
df.sort_index(inplace=True)

In [20]:
df.to_feather("../data/prev_next_click_for_2_columns")

# Create for 3 or more columns

In [14]:
import pandas as pd
import numpy as np
from logzero import logger
import gc

In [2]:
df = pd.read_feather("../data/time_table", nthreads=4)[["dow"]]

In [3]:
df["second"] = pd.read_feather("../data/time_of_day", nthreads=4)["second_of_day"]

In [4]:
df["epoctime"] = df["dow"]*24*3600+df["second"]
df.drop(columns=["second"], inplace=True)

In [5]:
df

,dow,epoctime
0,0,50549
1,0,50622
2,0,50660
3,0,50700
4,0,50460
5,0,50538
6,0,50616
7,0,50631
8,0,50642
9,0,50655


In [6]:
df = pd.concat([df, pd.read_feather("../data/multi_basic", nthreads=4)], axis=1, copy=False)

In [8]:
df.columns

Index(['dow', 'epoctime', 'ip_os', 'ip_device', 'ip_app', 'ip_channel',
       'os_device', 'os_app', 'os_channel', 'device_app', 'device_channel',
       'app_channel', 'ip_os_device', 'ip_os_app', 'ip_os_channel',
       'ip_device_app', 'ip_device_channel', 'ip_app_channel', 'os_device_app',
       'os_device_channel', 'os_app_channel', 'device_app_channel',
       'ip_os_device_app', 'ip_os_device_channel', 'ip_os_app_channel',
       'ip_device_app_channel', 'os_device_app_channel',
       'ip_os_device_app_channel'],
      dtype='object')

In [9]:
df.drop(columns=['ip_os', 'ip_device', 'ip_app', 'ip_channel',
       'os_device', 'os_app', 'os_channel', 'device_app', 'device_channel',
       'app_channel'], inplace=True)

In [10]:
columns = ['ip_os_device', 'ip_os_app', 'ip_os_channel',
       'ip_device_app', 'ip_device_channel', 'ip_app_channel', 'os_device_app',
       'os_device_channel', 'os_app_channel', 'device_app_channel',
       'ip_os_device_app', 'ip_os_device_channel', 'ip_os_app_channel',
       'ip_device_app_channel', 'os_device_app_channel',
       'ip_os_device_app_channel']
len(columns)

16

In [ ]:
for column in columns:
    logger.info(column)
    df = create_next_prev(df, column)

[I 180428 06:34:45 <ipython-input-15-398cd36da969>:2] ip_os_device
[I 180428 06:34:45 <ipython-input-11-ddf238345cf0>:2] sorting...
[I 180428 06:36:35 <ipython-input-11-ddf238345cf0>:4] sorted
[I 180428 06:36:42 <ipython-input-11-ddf238345cf0>:11] done prev
[I 180428 06:36:50 <ipython-input-11-ddf238345cf0>:16] done next
[I 180428 06:37:09 <ipython-input-11-ddf238345cf0>:19] dropped
[I 180428 06:37:09 <ipython-input-15-398cd36da969>:2] ip_os_app
[I 180428 06:37:09 <ipython-input-11-ddf238345cf0>:2] sorting...
[I 180428 06:39:33 <ipython-input-11-ddf238345cf0>:4] sorted
[I 180428 06:39:41 <ipython-input-11-ddf238345cf0>:11] done prev
[I 180428 06:39:49 <ipython-input-11-ddf238345cf0>:16] done next
[I 180428 06:40:12 <ipython-input-11-ddf238345cf0>:19] dropped
[I 180428 06:40:12 <ipython-input-15-398cd36da969>:2] ip_os_channel
[I 180428 06:40:12 <ipython-input-11-ddf238345cf0>:2] sorting...
[I 180428 06:42:49 <ipython-input-11-ddf238345cf0>:4] sorted
[I 180428 06:42:57 <ipython-input-11-

In [16]:
df.columns

Index(['dow', 'epoctime', 'ip_os_device', 'ip_os_app', 'ip_os_channel',
       'ip_device_app', 'ip_device_channel', 'ip_app_channel', 'os_device_app',
       'os_device_channel', 'os_app_channel', 'device_app_channel',
       'ip_os_device_app', 'ip_os_device_channel', 'ip_os_app_channel',
       'ip_device_app_channel', 'os_device_app_channel',
       'ip_os_device_app_channel', 'prev_click_second_ip_os_device',
       'next_click_second_ip_os_device', 'prev_click_second_ip_os_app',
       'next_click_second_ip_os_app', 'prev_click_second_ip_os_channel',
       'next_click_second_ip_os_channel', 'prev_click_second_ip_device_app',
       'next_click_second_ip_device_app',
       'prev_click_second_ip_device_channel',
       'next_click_second_ip_device_channel',
       'prev_click_second_ip_app_channel', 'next_click_second_ip_app_channel',
       'prev_click_second_os_device_app', 'next_click_second_os_device_app',
       'prev_click_second_os_device_channel',
       'next_click_secon

# a

In [1]:
import pandas as pd

In [2]:
df = pd.read_feather("../data/prev_next_click_for_3_columns", nthreads=4)

In [3]:
df.columns

Index(['dow', 'epoctime', 'ip_os_device', 'ip_os_app', 'ip_os_channel',
       'ip_device_app', 'ip_device_channel', 'ip_app_channel', 'os_device_app',
       'os_device_channel', 'os_app_channel', 'device_app_channel',
       'ip_os_device_app', 'ip_os_device_channel', 'ip_os_app_channel',
       'ip_device_app_channel', 'os_device_app_channel',
       'ip_os_device_app_channel', 'prev_click_second_ip_os_device',
       'next_click_second_ip_os_device', 'prev_click_second_ip_os_app',
       'next_click_second_ip_os_app', 'prev_click_second_ip_os_channel',
       'next_click_second_ip_os_channel', 'prev_click_second_ip_device_app',
       'next_click_second_ip_device_app',
       'prev_click_second_ip_device_channel',
       'next_click_second_ip_device_channel',
       'prev_click_second_ip_app_channel', 'next_click_second_ip_app_channel',
       'prev_click_second_os_device_app', 'next_click_second_os_device_app',
       'prev_click_second_os_device_channel',
       'next_click_secon

In [4]:
df.drop(columns=['dow', 'epoctime', 'ip_os_device', 'ip_os_app', 'ip_os_channel',
       'ip_device_app', 'ip_device_channel', 'ip_app_channel', 'os_device_app',
       'os_device_channel', 'os_app_channel', 'device_app_channel',
       'ip_os_device_app', 'ip_os_device_channel', 'ip_os_app_channel',
       'ip_device_app_channel', 'os_device_app_channel',
       'ip_os_device_app_channel'], inplace=True)

In [6]:
df.to_feather("../data/prev_next_click_for_3_columns")